<a href="https://colab.research.google.com/github/DanDas3/reconhecimento-digitos/blob/master/digit_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#coding: utf-8
import cv2 as cv
import numpy as np
import os
import keras #Rede Neural
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from matplotlib import pyplot as plt
from getpass import getpass
from os import listdir
from os.path import isfile, join

In [0]:
root = "reconhecimento-digitos"
data_root = "data"
out_path_root = "output"
digits_trains = "cvl-single-digits-train-validation/train"
digits_eval = "cvl-single-digits-train-validation/valid"
digits_valid = "cvl-single-digits-completeDatabase/cvl-single-digits/valid/"
string_trains = "cvl-strings-train/train"
string_eval = "cvl-strings-eval/cvl-strings-eval"

In [3]:
if os.path.exists(root) == False:
  !git clone https://github.com/DanDas3/reconhecimento-digitos.git

Cloning into 'reconhecimento-digitos'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 79494 (delta 17), reused 1 (delta 0), pack-reused 79445
Receiving objects: 100% (79494/79494), 187.79 MiB | 38.73 MiB/s, done.
Resolving deltas: 100% (21/21), done.
Checking out files: 100% (93528/93528), done.


In [0]:
# Funções

def convertBin(img):
  max_value = 255
  neighborhood_size=99
  subtract_from_mean = 10
  return cv.adaptiveThreshold(img, max_value, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, neighborhood_size, subtract_from_mean)
  #return cv.threshold(img, 200, 255, cv.THRESH_BINARY)

def segmenta(img):
  linha = img.shape[0]
  coluna = img.shape[1]
  
  # armazena uma tupla com o inicio e fim do dígito
  posicoes = []

  inicioBool = False
  #fimBool = False
  xInicial = 0
  xFinal = 0
  countPreto = 0

  for x in range(coluna):
    for y in range(linha):
      if(pb[y][x] == 0):
        if (inicioBool == False):
          inicioBool = True
          xInicial = x;

        countPreto += 1
    if(countPreto == 0 and inicioBool == True):
        xFinal = x
        inicioBool = False
        posicoes.append(((xInicial,0+5), (xFinal, pb.shape[0]-5)))  
    countPreto = 0
  return posicoes

def marcarSegmentos(img, positions):
  for position in positions:
    cv.rectangle(img, position[0],position[1],(0,0,0),thickness=2)
  return img

def carregaDigitTrain():
  path = root + "/" + data_root + "/" + digits_trains + "/"
  images = [f for f in listdir(path) if isfile(join(path, f))]
  data = np.ndarray((len(images), 32, 32, 3))
  tags = np.ndarray((len(images), 1))
  for i in range(len(images)):
    img = cv.imread(path + images[i])
    h_medio += img.shape[0]
    w_medio += img.shape[1]
    img = cv.resize(img,(32,32))
    tags[i] = int(images[i][:1])
    data[i] = img
  return data, tags

def carregaDigitEval():
  path = root + "/" + data_root + "/" + digits_eval + "/"
  images = [f for f in listdir(path) if isfile(join(path, f))]
  data = np.ndarray((len(images), 32, 32, 3))
  tags = np.ndarray((len(images), 1))
  for i in range(len(images)):
    img = cv.imread(path + images[i])
    img = cv.resize(img,(32,32))
    tags[i] = int(images[i][:1])
    data[i] = img
  
  return data, tags

def carregaDigitValid():
  path = root + "/" + data_root + "/" + digits_valid + "/"
  images = [f for f in listdir(path) if isfile(join(path, f))]
  data = np.ndarray((len(images), 32, 32, 3))
  tags = np.ndarray((len(images), 1))
  for i in range(len(images)):
    img = cv.imread(path + images[i])
    img = cv.resize(img,(32,32))
    tags[i] = int(images[i][:1])
    data[i] = img
  return data, tags

def carregaStringsValid():
  path = root + "/" + data_root + "/" + string_eval + "/"
  images = [f for f in listdir(path) if isfile(join(path, f))]
  data = np.ndarray((len(images), 32, 32, 3))
  tags = np.ndarray((len(images), 1))
  for i in range(len(images)):
    img = cv.imread(path + images[i])
    img = cv.resize(img,(365,89))    
    tag_num = images[i].split("-",1)[0]
    tags[i] = int(tag_num)
    data[i] = img
  
  return data, tags

In [0]:
#Rede Neural
from keras.datasets import imdb
from keras.preprocessing.text import Tokenizer
from keras import models
from keras import layers

# Set random seed
np.random.seed(0)

# Set the number of features we want
number_of_features = 1000

# Load data and target vector from movie review data
(data_train, target_train), (data_test, target_test) = imdb.load_data(
num_words=number_of_features)

In [0]:
# Divide em treinamento, teste e validação
(data_train, data_valid) = data_train[number_of_features//2:], data_train[:number_of_features//2]
(target_train, target_valid) = target_train[number_of_features//2:], target_train[:number_of_features//2]
# Convert movie review data to one-hot encoded feature matrix
tokenizer = Tokenizer(num_words=number_of_features//2)
features_train = tokenizer.sequences_to_matrix(data_train, mode="binary")
features_valid = tokenizer.sequences_to_matrix(data_valid, mode="binary")

data_test = data_test[:500]
target_test = target_test[:500]
tokenizer = Tokenizer(num_words=number_of_features//2)
features_test = tokenizer.sequences_to_matrix(data_test, mode="binary")

In [0]:
# Start neural network
network = models.Sequential()

# Add fully connected layer with a ReLU activation function
network.add(layers.Dense(units=16, activation="relu", input_shape=(number_of_features//2,)))

# Add fully connected layer with a ReLU activation function
network.add(layers.Dense(units=16, activation="relu"))

# Add fully connected layer with a sigmoid activation function
network.add(layers.Dense(units=1, activation="sigmoid"))

In [0]:
# Compile neural network
network.compile(loss="binary_crossentropy", # Cross-entropy
optimizer="rmsprop", # Root Mean Square Propagation
metrics=["accuracy"]) # Accuracy performance metric

In [30]:
# Train neural network

from keras.callbacks import ModelCheckpoint   

# Salva o melhor resultado
checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose=1, save_best_only=True)

history = network.fit(features_train, # Features
target_train, # Target vector
epochs=20, # Number of epochs
verbose=2, # Print description after each epoch
batch_size=10, # Number of observations per batch
validation_data=(features_valid, target_valid), # Validation data
callbacks=[checkpointer],
shuffle=True)

Train on 24500 samples, validate on 500 samples
Epoch 1/20
 - 7s - loss: 0.3699 - acc: 0.8433 - val_loss: 0.3579 - val_acc: 0.8420

Epoch 00001: val_loss improved from inf to 0.35786, saving model to model.weights.best.hdf5
Epoch 2/20
 - 7s - loss: 0.3627 - acc: 0.8460 - val_loss: 0.3538 - val_acc: 0.8320

Epoch 00002: val_loss improved from 0.35786 to 0.35381, saving model to model.weights.best.hdf5
Epoch 3/20
 - 8s - loss: 0.3547 - acc: 0.8524 - val_loss: 0.3785 - val_acc: 0.8400

Epoch 00003: val_loss did not improve from 0.35381
Epoch 4/20
 - 8s - loss: 0.3470 - acc: 0.8577 - val_loss: 0.3731 - val_acc: 0.8340

Epoch 00004: val_loss did not improve from 0.35381
Epoch 5/20
 - 8s - loss: 0.3397 - acc: 0.8637 - val_loss: 0.4094 - val_acc: 0.8420

Epoch 00005: val_loss did not improve from 0.35381
Epoch 6/20
 - 8s - loss: 0.3330 - acc: 0.8682 - val_loss: 0.3988 - val_acc: 0.8240

Epoch 00006: val_loss did not improve from 0.35381
Epoch 7/20
 - 7s - loss: 0.3242 - acc: 0.8719 - val_loss

In [0]:
# Carrega os pesos com melho precisão na validação
network.load_weights('model.weights.best.hdf5')

In [53]:
print(features_test.shape)
print(features_test[0].shape)
print(features_test[0:1].shape)

(500, 500)
(500,)
(1, 500)


In [57]:
# Avaliaçãão do treinamento
score = network.evaluate(features_test, target_test, verbose=0)
print("\n","Resultado:", score[1])


 Resultado: 0.8400000009536743


In [0]:
y_hat = network.predict(features_test)

In [65]:
print(type(data_test[0]))
print(type(target_test[0]))
print(target_test[2])
for i, idx in enumerate(np.random.choice(features_test.shape[0], size=32, replace=False)):
  pred_idx = np.argmax(y_hat[idx])
  true_idx = np.argmax(data_test[idx])

  print(data_test[true_idx] == data_test[pred_idx] )

<class 'list'>
<class 'numpy.int64'>
1
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
